<a href="https://colab.research.google.com/github/JulianAkkerman/Inspiratie-/blob/main/Kopie_van_M4LP_Assignment_2_(2025)_student_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment 2

This is the complete Assignment 2. You are asked to train and test linear and logistic regression models and access lexical resources.

# <font color="red">Contributions</font>

~~Delete this text and write instead of it your:~~
* ~~group number (same as the file name, for sanity chack)~~
* ~~a list of group members names (NOT student IDs)~~
* ~~who contributed to which exercises (you don't need to be very detailed)~~

To start the assignment, import prerequisite packages:

In [ ]:
pip install torchtext==0.15.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.

In [ ]:
import torch
import torch.nn.functional as F
import torchtext as text
import numpy as np
from tqdm.notebook import trange, tqdm
import nltk,sklearn
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
import collections, itertools
import more_itertools

This is all you need. **Do not import anything else at other points** except lexical resources WordNet and FrameNet where suggested.
To do this assignment, you are expected to refer to pyTorch and torchtext documentation.

# 2.1 Import GloVe word embedding model

GloVe contains _static_ word embeddings. This means that the vector is assigned to word types and does not vary in different contexts or for different word senses. Pretrained GloVe word embedding models exist in different sizes. For the purpose of the exercise, we will use the smallest GloVe vectors with 50 dimensions. First, let's download the vectors. This may take a few minutes.

In [ ]:
glovedim=50
#If you experience an issue downloading the vectors, try a different download source by uncommenting the following line:
#text.vocab.GloVe.url["6B"] = "https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip"
vec = text.vocab.GloVe(name='6B', dim=glovedim)

.vector_cache/glove.6B.zip: 862MB [02:45, 5.22MB/s]                           
100%|█████████▉| 399999/400000 [00:11<00:00, 34181.44it/s]


In [ ]:
type(vec.dim)

int

**Exercise.** How many words do the GloVe embeddings contain?

In [ ]:
print(vec.__len__())


400000


**Exercise**. We can now check if particular orthographic words have vectors in `vec`. Check if strings 'asdfdfasd', 'catc', 'cact', and 'cat' have a glove vector, and print the resuts

In [ ]:
tokens_list = ['asdfdfasd', 'catc', 'cact', 'cat']
tokens_vector = vec.get_vecs_by_tokens(tokens_list)

for vector in tokens_vector:
    print(vector)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])
tensor([-0.9832, -0.5770, -0.4891, -0.2312, -0.7746, -0.2599,  0.8804, -0.6786,
         0.0752, -0.6429,  0.2577,  0.2738,  0.5285,  0.2630, -0.7031,  0.3494,
        -0.2407,  0.7581,  0.2753,  0.3231, -0.1638, -0.4644, -0.5918, -0.1965,
         0.0926,  0.5420,  0.5445, -0.2751, -0.2848,  0.0364, -0.7616, -0.5728,
         0.0859,  0.6841, -0.4170,  0.6888,  0.0200,  0.2174, -0.1152, -0.0930,
         0.5504, -0.5972,  0.2634,  0.6699,  0.3886, -0.1229,  1.0426, -0.0213,
        -0.4894,  0.1489])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])
tensor([ 0.4528, -0.5011, -0.5371, -0.0157,

**Exercise**. What is the vector of _cat_?



In [ ]:
cat_vector = vec.get_vecs_by_tokens('cat')
print(cat_vector)

tensor([ 0.4528, -0.5011, -0.5371, -0.0157,  0.2219,  0.5460, -0.6730, -0.6891,
         0.6349, -0.1973,  0.3368,  0.7735,  0.9009,  0.3849,  0.3837,  0.2657,
        -0.0806,  0.6109, -1.2894, -0.2231, -0.6158,  0.2170,  0.3561,  0.4450,
         0.6089, -1.1633, -1.1579,  0.3612,  0.1047, -0.7832,  1.4352,  0.1863,
        -0.2611,  0.8328, -0.2312,  0.3248,  0.1449, -0.4455,  0.3350, -0.9595,
        -0.0975,  0.4814, -0.4335,  0.6945,  0.9104, -0.2817,  0.4164, -1.2609,
         0.7128,  0.2378])


# 2.2 Linear regression: Concreteness prediction

Obtain concreteness ratings from the paper

Brysbaert, M., Warriner, A., & Kuperman, V. (2014). Concreteness ratings for 40 thousand generally known English word lemmas. BEHAVIOR RESEARCH METHODS, 46 (3), 904–911. https://doi.org/10.3758/s13428-013-0403-5

In [ ]:
!wget https://raw.githubusercontent.com/ArtsEngine/concreteness/master/Concreteness_ratings_Brysbaert_et_al_BRM.txt

--2025-02-09 12:41:12--  https://raw.githubusercontent.com/ArtsEngine/concreteness/master/Concreteness_ratings_Brysbaert_et_al_BRM.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1646191 (1.6M) [text/plain]
Saving to: ‘Concreteness_ratings_Brysbaert_et_al_BRM.txt’

Concreteness_rating 100%[===================>]   1.57M  --.-KB/s    in 0.05s   

2025-02-09 12:41:12 (29.4 MB/s) - ‘Concreteness_ratings_Brysbaert_et_al_BRM.txt’ saved [1646191/1646191]



This is a tab-separated file with a header. Structured data like this can be conveniently read via DictReader class from csv.

In [ ]:
import csv

Using DictReader, create lists ```concreteness_words``` and ```concreteness_scores``` of words in the concreteness ratings file that have a GloVe vector

In [ ]:
concreteness_words=[]
concreteness_scores=[]
with open("Concreteness_ratings_Brysbaert_et_al_BRM.txt",'r') as concfile:
  read_tsv = csv.DictReader(concfile, delimiter="\t")
  # complete the code below
  for row in read_tsv:
    word=row["Word"]
    if word in vec.stoi:
      concreteness_words.append(word)
      concreteness_scores.append(float(row["Conc.M"]))

How many words ended up in your concreteness dataset?

In [ ]:
len(concreteness_words)

31618

Create train and test partitions of the concreteness data:

In [ ]:
conc_words_train, conc_words_test, conc_scores_train, conc_scores_test = train_test_split(concreteness_words,concreteness_scores,test_size=0.1)

Convert data to torch tensor format to use in a regression model:

In [ ]:
vecs_train=torch.stack([vec[w] for w in conc_words_train])
vecs_test=torch.stack([vec[w] for w in conc_words_test])
scores_train=torch.tensor(conc_scores_train)
scores_test=torch.tensor(conc_scores_test)

**Exercise** Explain in a couple of sentences what the second and third lines of code in the above cell does.
2. Lijn 2 creert 1 tensor object door alle vectoren die bij de test subset horen in een lijst te zetten en die lijst wordt vervolgens
    omgezet in een tensor object door middel van stack.

3. Lijn 3 zet de train concreteness scores om in een tensor object.
YOUR EXPLANATION HERE

Now we can define linear regression model in pyTorch:

In [ ]:
class Regression(torch.nn.Module):
     def __init__(self, input_dim, output_dim):
         super(Regression, self).__init__()
         self.linear = torch.nn.Linear(input_dim, output_dim)
     def forward(self, x):
         outputs = self.linear(x)
         return outputs

**Exercise**. Explain each line of the class definition above. For example, line 3 can be explained as follows:

`super(Regression, self).__init__()` initializes a new Regression model as an instance of the parent class `torch.nn.Module`

Now explain all other lines of the class code.

A specific linear regression model can have the input dimensionality of our word embeddings and 1-dimensional input (the concretenss score)

In [ ]:
model = Regression(glovedim,1)

To train the model, we need a loss function and an optimiser, including the learning rate parameter. We choose Mean Square Error loss and Stochastic Gradient Descent method with the learning rate of 0.1.

In [ ]:
criterion = torch.nn.MSELoss()

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

We can now train our linear regression model using gradient descent.

**Exercise**. Complete the code below. After every 5 training steps, evaluate the model, printing out loss and accuracy for the training and test sets. Calculate the accuracy as the percentage of examples where the predicted score of the model differs from the correct score by less than 1. The training may take a minute or so.

In [ ]:
losses = []
losses_test = []
Iterations = []
epochs = 5000
tolerance = 1.0
#after defining parameters, we train the model several times on the same data; each iteration is an epoch:
for epoch in trange(epochs, desc='Training Epochs'):
    x = vecs_train
    scores = scores_train
    optimizer.zero_grad()
    #here we pass the word vectors from the training set, obtaining regression model's predicted outputs:
    outputs = model(x)
    loss = criterion(torch.squeeze(outputs), scores)
    loss.backward()
    optimizer.step()

    #Now, every 5 epochs we can evaluate how the model performs on the data
    if (epoch + 1) % 5 == 0:
        #we don't compute gradients as the model is only evaluated and not updated

        with torch.no_grad():
            model.eval()
            #complete the code for evaluating the model here
            print(f"loss: {loss}")

            correct = 0

            outputs_test = model(vecs_test)

            for i in range (outputs_test.__len__()):
                if (outputs_test[i]- scores_test[i] <1):
                    correct+=1


            print(f"accuracy: {correct/outputs_test.__len__()}")






Training Epochs:   0%|          | 0/5000 [00:00<?, ?it/s]

loss: 0.661749541759491
accuracy: 0.9320050600885516
loss: 0.6053734421730042
accuracy: 0.9354838709677419
loss: 0.5648090839385986
accuracy: 0.9380139152435167
loss: 0.535064697265625
accuracy: 0.9392789373814042
loss: 0.5130048990249634
accuracy: 0.9402277039848197
loss: 0.49652299284935
accuracy: 0.9402277039848197
loss: 0.4841454327106476
accuracy: 0.9414927261227072
loss: 0.4748154580593109
accuracy: 0.9414927261227072
loss: 0.46776294708251953
accuracy: 0.9411764705882353
loss: 0.4624202251434326
accuracy: 0.9408602150537635
loss: 0.45836561918258667
accuracy: 0.9411764705882353
loss: 0.4552839994430542
accuracy: 0.9411764705882353
loss: 0.45293885469436646
accuracy: 0.9408602150537635
loss: 0.45115208625793457
accuracy: 0.9414927261227072
loss: 0.4497891962528229
accuracy: 0.9408602150537635
loss: 0.448748379945755
accuracy: 0.9405439595192916
loss: 0.4479525685310364
accuracy: 0.9399114484503479
loss: 0.44734323024749756
accuracy: 0.939595192915876
loss: 0.4468758702278137
accu

**Exercise**. What are the predicted concreteness scores of the nouns _logarithm_ and _mouse_?

In [ ]:
def predicted_concreteness(wd):
  #complete the code here

  vec_wd = vec.get_vecs_by_tokens(wd)
  model_train = model(vec_wd)



  print(float(model_train))

predicted_concreteness('logarithm')
predicted_concreteness("mouse")




2.7750699520111084
4.64422082901001


# 2.3. Create a dataset of WordNet supersenses for words that have GloVe vectors.

First, download the WordNet database:

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


You can read documentation on object types in WordNet, for example by invoking the types:

In [ ]:
nltk.corpus.reader.wordnet.Lemma

nltk.corpus.reader.wordnet.Lemma

In [ ]:
nltk.corpus.reader.wordnet.Synset

nltk.corpus.reader.wordnet.Synset

**Exercise**. Now, retrieve the first lemma corresponding to the adjective _dry_ as `d`:

In [ ]:
synset = wn.synsets("dry")

d= synset[0].lemmas()[0]
print(d)


SyntaxError: invalid syntax (<ipython-input-23-26619455e7fa>, line 1)

What lemmas belong to the same SynSet as `d`? Retrieve the list.

In [ ]:
#your code here

dry_lemmas = [lemma  for lemma in d.synset().lemmas()]

print(dry_lemmas)


**Exercise**. Define function `ant_freq` that returns the frequency of (the first) antonym of a lemma. The frequency value should be retrieved from  WordNet .

In [ ]:
def ant_freq(x):
  #your code here

SyntaxError: incomplete input (<ipython-input-36-614f912c3ab6>, line 2)

Apply `ant_freq` to `d`. This will output the frequency of _wet_.

In [ ]:
ant_freq(d)

NameError: name 'ant_freq' is not defined

**Exercise**. Now, create a dataset that includes for each word in WordNet that has a GloVe vector the lexicographic file (supersense) of its first synset.

In [ ]:
#your code here
wn_words =
wn_supersenses =

SyntaxError: invalid syntax (<ipython-input-27-aacc66e8c283>, line 2)

Split the dataset into train and test partitions:

In [ ]:
wn_words_train, wn_words_test, wn_supersenses_train, wn_supersenses_test = train_test_split(wn_words,wn_supersenses,test_size=0.1)

NameError: name 'wn_words' is not defined

# 2.4. Logistic regression: word class prediction.

Now we can address a classification task. Define a (multinomial) regression model using softmax, choose a loss function and an optimizer for it.

**Exercise**. Initialize your model. Use the same Regression class for logistic regression as for linear regression - the difference will come from the objective (loss) function. The output now is not a single number but scores for each of the classes in the classification task. Estimate `num_classes`prior to initializing the model

In [ ]:
num_classes =
logreg_model = Regression(glovedim,num_classes)

Choose the loss function. This is a crucial choice: some of the loss functions in PyTorch (see https://pytorch.org/docs/stable/nn.html) already include a softmax or a sigmoid in their implementaion, which gives computational advantages. Be sure to read the documentation on your loss function to confirm you made a good choice.

In [ ]:
#your code here


And initialize the optimiser:

In [ ]:
#your code here


WordNet is quite big. For efficiency, use the following function for splitting your data into batches when processing:

In [ ]:
batch_size = 200
def get_batches(src_iter, tgt_iter, batch_size=batch_size):
    for batch in more_itertools.chunked(zip(src_iter, tgt_iter), batch_size):
        x, y = zip(*batch)
        x = torch.stack(x)
        y = torch.stack(y)
        yield x, y

**Exercise**. Train and test your logistic regression model, printing the train and test loss and accuracy:

In [ ]:
#your code here


Define a mapping from indices to lexicographic file names:

In [ ]:
#your code
itolexname=

What is the name of lexicographic file 2?

In [ ]:
itolexname[2]

**Exercise**. Which supersense (lexicographic file) does your classifier assign to the noun _house_?

In [ ]:
def predicted_lexname(wd):
#your code
#

predicted_lexname("house")

Which supersense (lexicographic file) does your classifier assign to the noun _hog_?

In [ ]:
predicted_lexname("hog")

# 2.5. Hypernymy classification

Now, download a lexical entailment (hypernymy) dataset called WBLESS. The dataset was developed by Weeds et al. with the goal of testing models on distinguishing hypernyms from other related word pairs.

Weeds et al. (2014) Julie Weeds, Daoud Clarke, Jeremy Reffin, David Weir, and Bill Keller. 2014. Learning to distinguish hypernyms and co-hyponyms. In Proceedings of the 2014 International Conference on Computational Linguistics, pages 2249–2259, Dublin, Ireland.

WBLESS (together with other relevant datasets) can be conveniently downloaded from the Facebook Research github page by Stephen Roller who worked on hypernymy learning:

Stephen Roller, Douwe Kiela, and Maximilian Nickel. 2018. Hearst Patterns Revisited: Automatic Hypernym Detection from Large Text Corpora. ACL.

Download the tab-separated dataset:

In [ ]:
!wget https://github.com/facebookresearch/hypernymysuite/raw/main/data/wbless.tsv

Check how the data looks:

In [ ]:
!head wbless.tsv

We used ```csv``` above to process a tab separated file. It can also be done using ```pandas```:

In [ ]:
wbless_df = pd.read_csv('wbless.tsv', sep='\t')
wbless_df.head()

,word1,word2,label,relation,fold
0,frigate,craft,True,hyper,val
1,trouble,carp,False,other,val
2,fox,mouth,False,other,val
3,foot,robin,False,other,val
4,vest,garment,True,hyper,val


Now create training and test data for relation classification:

In [ ]:
#your code here

Split into training and test data:

In [ ]:
wbless_words_train, wbless_words_test, hypernymy_train, hypernymy_test = train_test_split(wbless_words,hypernymy,test_size=0.1)

**Exercise**. Create, train and test a logistic regression model that predicts whether two words stand in the hypernymy relation given their GloVe vectors.

Make sure your model predicts a single score used for the binary decision (hypernymy vs. non-hypernymy) rather than scores for multiple classes, and choose the loss function in pyTorch accordingly.

Print the train and test loss and accuracy. Use the concatenation of the two words' vectors as input to the logistic regression classifier.

In [ ]:
#your code here


What label does your model predict for the pair _dog,animal_? Your code below should produce a Boolean value, `True` for the positive class (hypernymy) and `False` for the negative class (not hypernymy).

In [ ]:
def predicted_hypernymy(w1,w2):
  #complete the code

predicted_hypernymy("dog","animal")

What label does your model predict for the pair _dog,cat_?

In [ ]:
predicted_hypernymy("dog","cat")

What label does your model predict for the pair _animal,dog_?

In [ ]:
predicted_hypernymy("animal","dog")

#2.6 Using FrameNet



You can explore FrameNet online:
https://framenet.icsi.berkeley.edu/frameIndex

Or read detailed documentation here:
https://framenet2.icsi.berkeley.edu/docs/r1.7/book.pdf

Now, load FrameNet via the NLTK package:

In [ ]:
nltk.download('framenet_v17')
from nltk.corpus import framenet as fn

`fn.lus` allows to retrieve lexical units (LUs) recorded in FrameNet. A lexical unit approximately corresponds to a lemma in WordNet, i.e. a word taken in a specific sense. Without additional parameters, it returns a complete list:



In [ ]:
fn.lus()

[<lu ID=16601 name=(can't) help.v>, <lu ID=14632 name=(in/out of) line.n>, ...]

You can also retrieve lexical units by regular expression search. For example, the following returns the list of LUs that contain string _pres_ at the beginning of the LU name:

In [ ]:
fn.lus('^pres')

[<lu ID=957 name=press.v>, <lu ID=10117 name=press.v>, ...]

Individual lexical units can be retrieved by ID, for example:

In [ ]:
fn.lu(10117)

lexical unit (10117): press.v

[definition]
  COD: make strong efforts to persuade or force to do something.

[frame] Attempt_suasion(87)

[POS] V

[status] Finished_Initial

[lexemes] press/V

[semTypes] 0 semantic types

[URL] https://framenet2.icsi.berkeley.edu/fnReports/data/lu/lu10117.xml

[subCorpus] 12 subcorpora
  01-T-Won-(1), 02-T-Wto-(1), 03-T-Winto-(1), 04-T-Wwith-(1),
  05-T-Wfor-(1), 06-AVP-T-(1), 07-T-AVP-(1), 08-T-NP-PP-(1),
  09-T-NP-(1), manually-added, other-matched-(1), other-
  unmatched-(1)

[exemplars] 20 sentences across all subcorpora

Terms that show up in square brackets are attributes, in this case attributes of the lexical unit. They include usage examples:

In [ ]:
fn.lu(10117).exemplars

exemplar sentences for press.v in Attempt_suasion:

[0] The sponsors of the bill made clear their intention to press for a vote on it within the current legislative session .
[1] Anne McIntosh , MEP for North East Essex , is pressing the Government to reverse its policy of controlled retreat , which allows the sea to gradually take its natural course .
[2] Lewis pressed Adam to accompany him on those solicitous weekend visits but Adam nearly always said he was too busy or would be bored .
[3] The committee also agreed to press for changes to the rule which limits the level of right-to-buy discounts in the case of newly built houses or those recently modernised .
[4] It is no secret that Damascus has been using the PKK as a bargaining chip to press Turkey into complying with its demands for more water from the Euphrates , on which it heavily depends .
[5] The tenant 's adviser should therefore press for an obligation on the landlord to notify the tenant of any application made by him to

... and the frame that the lexical unit evokes, which in turn has its own attributes:

In [ ]:
fn.lu(10117).frame

frame (87): Attempt_suasion

[URL] https://framenet2.icsi.berkeley.edu/fnReports/data/frame/Attempt_suasion.xml

[definition]
  The  Speaker expresses through language his wish to get the
  Addressee to act in some way that will help to bring about events
  or states described in the Content. There is no implication that
  the Addressee forms an intention to act, let alone acts.     'Mr
  Smithers always encourages the employees to stay late and work
  harder.'  'Dennis Rodman advises moderation in all things. INI'
  The Content most prototypically refers to an action that the
  Addressee will carry out themselves, but may (in the case of
  valences with a non-finite Content clause) merely refer to a
  situation that they have indirect influence over, as in the
  following  'When I talked to her, I suggested that he be removed
  from office . DNI '

[semTypes] 0 semantic types

[frameRelations] 8 frame relations
  <Parent=Attempt -- Inheritance -> Child=Attempt_suasion>
  <Parent=Attem

**Exercise**. Find all LUs on FrameNet that share the frame with the noun *car*. Print these words along with their definitions.

In [ ]:
#your code here

**Exercise**. Define a function that takes a FrameNet frame as input and prints out the definitions of all core frame elements associated with the frame. For example for the frame associated with the noun _car_ your function will print the definition of the only core frame element _Vehicle_:


> Vehicle is the transportation device that the human beings use to travel.This FE is incorporated into each LU in this frame.

In [ ]:
def printcoreFE(frame):
  """
  Args:
    frame: a frame object from FrameNet
  """
#your code here

Test your function on the frame evoked by the verb _sing_:

In [ ]:
sing=
printcoreFE(sing)